In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.keras import Model, layers, Input, optimizers, Sequential, optimizers

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
try:
    for gpu in gpus:
        print(gpu)
        tf.config.experimental.set_memory_growth(gpu, True)
except RuntimeError as e:
    print(e)

In [3]:
from data_utils import generate_music, load_music_utils

Using TensorFlow backend.


In [4]:
X, Y, n_values, indices_values = load_music_utils()
print('shape of X:', X.shape)  # 60段音乐  30个时间步 每个时间步输入为78个值中的一个
print('number of training examples:', X.shape[0])
print('Tx (length of sequence):', X.shape[1])
print('total # of unique values:', n_values)
print('Shape of Y:', Y.shape)  # Ty,m 78

shape of X: (60, 30, 78)
number of training examples: 60
Tx (length of sequence): 30
total # of unique values: 78
Shape of Y: (30, 60, 78)


In [5]:
Y_ = np.rollaxis(Y, 1, 0)
Y_.shape

(60, 30, 78)

In [6]:
X_ = tf.cast(X, dtype='float32')
y_ = tf.cast(Y_, dtype='float32')

In [7]:
n_values = 78
n_a = 64
Ty = 100

In [8]:
LSTM = layers.LSTM(n_a, return_sequences=True)
densor = layers.Dense(n_values, activation='softmax')

In [9]:
def train(Tx, n_a, n_values):
    X = Input(shape=(Tx, n_values))
    out = LSTM(X)
    out =densor(out)
    model = Model(inputs=X, outputs=out)
    return model

In [10]:
train_model = train(30, n_a, n_values)

In [11]:
opt = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)

train_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
train_model.fit(X_, y_, epochs=100)

Train on 60 samples
Epoch 1/100
60/60 [==============================] - 3s 56ms/sample - loss: 4.1960 - accuracy: 0.0422
Epoch 2/100
60/60 [==============================] - 0s 1ms/sample - loss: 4.0644 - accuracy: 0.1122
Epoch 3/100
60/60 [==============================] - 0s 1ms/sample - loss: 3.8541 - accuracy: 0.0844
Epoch 4/100
60/60 [==============================] - 0s 781us/sample - loss: 3.7556 - accuracy: 0.0989
Epoch 5/100
60/60 [==============================] - 0s 781us/sample - loss: 3.6608 - accuracy: 0.1144
Epoch 6/100
60/60 [==============================] - 0s 1ms/sample - loss: 3.5933 - accuracy: 0.1322
Epoch 7/100
60/60 [==============================] - 0s 1ms/sample - loss: 3.5027 - accuracy: 0.1422
Epoch 8/100
60/60 [==============================] - 0s 1ms/sample - loss: 3.3923 - accuracy: 0.1450
Epoch 9/100
60/60 [==============================] - 0s 1ms/sample - loss: 3.2644 - accuracy: 0.1661
Epoch 10/100
60/60 [==============================] - 0s 1ms/sampl

Epoch 81/100
60/60 [==============================] - 0s 781us/sample - loss: 0.2325 - accuracy: 0.9222
Epoch 82/100
60/60 [==============================] - 0s 1ms/sample - loss: 0.2304 - accuracy: 0.9222
Epoch 83/100
60/60 [==============================] - 0s 1ms/sample - loss: 0.2285 - accuracy: 0.9222
Epoch 84/100
60/60 [==============================] - 0s 781us/sample - loss: 0.2266 - accuracy: 0.9222
Epoch 85/100
60/60 [==============================] - 0s 781us/sample - loss: 0.2248 - accuracy: 0.9222
Epoch 86/100
60/60 [==============================] - 0s 1ms/sample - loss: 0.2231 - accuracy: 0.9222
Epoch 87/100
60/60 [==============================] - 0s 1ms/sample - loss: 0.2214 - accuracy: 0.9222
Epoch 88/100
60/60 [==============================] - 0s 1ms/sample - loss: 0.2198 - accuracy: 0.9222
Epoch 89/100
60/60 [==============================] - 0s 1ms/sample - loss: 0.2182 - accuracy: 0.9222
Epoch 90/100
60/60 [==============================] - 0s 1ms/sample - loss: 

In [13]:
def one_hot(x):
    x = tf.argmax(x, axis=-1)
    x = tf.one_hot(x, 78) 
    x = layers.RepeatVector(1)(x)
    return x

In [14]:
x = tf.zeros((1, 78))
one_hot(x)

<tf.Tensor: id=4310, shape=(1, 1, 78), dtype=float32, numpy=
array([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]],
      dtype=float32)>

In [15]:
def generate_model(n_values = 78, n_a = 64, Ty = 100):
    x0 = Input(shape=(1, n_values))
    a0 = Input(shape=(n_a,))
    c0 = Input(shape=(n_a,))
    outputs = []
    a = a0
    c = c0
    x = x0
    w = LSTM.get_weights()
    lstm = tf.keras.layers.LSTM(n_a, weights=w, return_state=True)
    for t in range(Ty):
        out, a, c = lstm(x, initial_state=(a, c))
        out = densor(out)
        outputs.append(out)
        x =  tf.keras.layers.Lambda(one_hot)(out) 
    inference_model = tf.keras.Model(inputs=[x0, a0, c0], outputs=outputs)
    return inference_model

In [16]:
music_model = generate_model(n_values, n_a, Ty)

In [17]:
x_initializer = tf.zeros((1, 1, 78))
a_initializer = tf.zeros((1, n_a))
c_initializer = tf.zeros((1, n_a))

In [18]:
out_stream = generate_music(music_model)

Predicting new values for different set of chords.
Generated 100 sounds using the predicted values for the set of chords ("1") and after pruning
Generated 100 sounds using the predicted values for the set of chords ("2") and after pruning
Generated 101 sounds using the predicted values for the set of chords ("3") and after pruning
Generated 101 sounds using the predicted values for the set of chords ("4") and after pruning
Generated 101 sounds using the predicted values for the set of chords ("5") and after pruning
Your generated music is saved in output/my_music.midi
